# MODEL ARCHITECTURE

# Individual Models

In [5]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class ActionModel(nn.Module):
    """
    A model for predicting actions within a specific range for a given action type in a RL game using PyTorch.
    This version is adapted to accept 3D input data of shape (21, 11, 16).

    Parameters:
    - input_shape (tuple): The shape of the input data, not including the batch size.
    - output_size (int): The number of possible actions to predict.
    """
    def __init__(self, input_shape=(1, 21, 11, 16), output_size=1):
        super(ActionModel, self).__init__()

        # Define the architecture to handle 3D input
        # PyTorch Conv3d expects input of shape (N, C, D, H, W), hence input_shape is adjusted accordingly
        self.conv1 = nn.Conv3d(input_shape[0], 32, kernel_size=(3, 3, 3), padding='same')
        self.pool1 = nn.MaxPool3d(kernel_size=(2, 2, 2))
        self.conv2 = nn.Conv3d(32, 64, kernel_size=(3, 3, 3), padding='same')
        self.pool2 = nn.MaxPool3d(kernel_size=(2, 2, 2))

        # Compute the flattened size after the convolutional and pooling layers
        self.flattened_size = self._get_conv_output(input_shape)

        # Dense layers
        self.fc1 = nn.Linear(self.flattened_size, 512)
        self.fc2 = nn.Linear(512, 256)
        self.output_layer = nn.Linear(256, output_size)

    def _get_conv_output(self, shape):
        """Helper function to compute the size of the flattened features after convolutional layers"""
        with torch.no_grad():
            input = torch.rand(1, *shape)
            output = self.pool2(self.conv2(self.pool1(self.conv1(input))))
            return int(np.prod(output.size()[1:]))

    def forward(self, x):
        x = self.pool1(F.relu(self.conv1(x)))
        x = self.pool2(F.relu(self.conv2(x)))
        x = x.view(-1, self.flattened_size)  # Flatten the output for dense layers
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        return self.output_layer(x)

# Reading in Data and Training Models

In [6]:
from tqdm import tqdm

def train_on_chunk(data_loader, model, optimizer, loss_fn, device, num_epochs=100):
    model.train()
    history = {'loss': [], 'accuracy': []}

    for epoch in range(num_epochs):
        running_loss = 0.0
        correct_predictions = 0
        total_predictions = 0

        progress_bar = tqdm(data_loader, desc=f"Epoch {epoch+1}/{num_epochs}", leave=False)

        for inputs, labels in progress_bar:
            inputs, labels = inputs.to(device), labels.to(device)
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = loss_fn(outputs, labels)
            loss.backward()
            optimizer.step()

            running_loss += loss.item()
            _, predicted = torch.max(outputs, 1)
            correct_predictions += (predicted == labels).sum().item()
            total_predictions += labels.size(0)

            # Optionally, update the progress bar with the current batch's loss
            progress_bar.set_postfix({'loss': loss.item()})

        epoch_loss = running_loss / len(data_loader)
        epoch_accuracy = correct_predictions / total_predictions

        history['loss'].append(epoch_loss)
        history['accuracy'].append(epoch_accuracy)

        print(f"Epoch {epoch+1}/{num_epochs}, Loss: {epoch_loss:.4f}, Accuracy: {epoch_accuracy:.4f}")

        # Early stopping condition based on the improvement over the last 5 epochs
        if len(history['accuracy']) >= 5:
            if history['accuracy'][-1] <= history['accuracy'][-5] + 0.001:
                print(f"Early stopping triggered at epoch {epoch+1}.")
                break

    return history

In [7]:
import pandas as pd
import torch
from torch.utils.data import DataLoader, TensorDataset
import os
import numpy as np

def map_labels(labels):
    # Adjust labels to be zero-indexed and normalized
    min_label = np.min(labels)
    max_label = np.max(labels)
    labels_normalized = labels - min_label
    return labels_normalized, max_label - min_label + 1

def get_data_loader_for_action_type(base_dir, action_type, batch_size=128):
    """
    Loads and preprocesses data for a specific action type, returning a DataLoader.

    Args:
    - base_dir (str): Base directory containing action_type subdirectories.
    - action_type (str): The specific action type directory to process.
    - batch_size (int): Batch size for the DataLoader.

    Returns:
    - A DataLoader ready for model training, and the output size based on the labels.
    """
    folder_path = os.path.join(base_dir, action_type)
    board_tensors_path = os.path.join(folder_path, 'board_tensors.csv.gzip')
    actions_path = os.path.join(folder_path, 'actions.csv.gzip')

    # Ensure data files exist
    if not os.path.exists(board_tensors_path) or not os.path.exists(actions_path):
        print(f"Data files missing in {action_type}, skipping...")
        return None, None

    # Load data
    board_tensors = pd.read_csv(board_tensors_path, compression='gzip')
    actions = pd.read_csv(actions_path, compression='gzip')

    # Normalize labels and reshape data
    labels, output_size = map_labels(actions.iloc[:, 0].values)  # Normalize labels
    reshaped_tensors = [torch.tensor(row.values, dtype=torch.float32).reshape((1, 21, 11, 16)) for _, row in board_tensors.iterrows()]

    X = torch.stack(reshaped_tensors)
    y = torch.tensor(labels, dtype=torch.long)

    # Create TensorDataset and DataLoader
    dataset = TensorDataset(X, y)
    data_loader = DataLoader(dataset, batch_size=batch_size, shuffle=True)

    return data_loader

In [ ]:
import pickle
# Overall run
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")  # Move the model to the specified device before training starts

# ONLY CONSIDER ACTIONS WHERE THE OUTPUT SIZE > 1, so there's actually a decision to be made

action_models = {
    'action_type_1': ActionModel(output_size=19),
    'action_type_3': ActionModel(output_size=72),
    'action_type_4': ActionModel(output_size=54),
    'action_type_5': ActionModel(output_size=54),
    'action_type_8': ActionModel(output_size=20),
    'action_type_10': ActionModel(output_size=5),
    'action_type_11': ActionModel(output_size=60),
}

optimizers = {
    action_type: torch.optim.Adam(model.parameters(), lr=0.001)
    for action_type, model in action_models.items()
}


base_dir = './data'

for sub_folders in os.listdir(base_dir):
    folder_path = os.path.join(base_dir, sub_folders)

    # Check if the current path is indeed a directory
    if os.path.isdir(folder_path):
        for action_type, model in action_models.items():
            optimizer = optimizers[action_type]
            loss_fn = torch.nn.CrossEntropyLoss()
            model.to(device)

            # Assume you have a function to get the correct DataLoader for each action type
            data_loader = get_data_loader_for_action_type(folder_path, action_type)
            print(data_loader)
            if data_loader:
              # Train each model using its corresponding data loader and optimizer
              avg_loss, accuracy = train_on_chunk(data_loader, model, optimizer, loss_fn, device)
        

for key, model in action_models.items(): 
    # Move model to CPU for saving
    model.to('cpu')
    # Save model's state dictionary
    model_path = f'saved_model_{key}.pkl'
    with open(model_path, 'wb') as f:
      pickle.dump(model, f)

Using device: cuda


Epoch 1/100, Loss: 2.5868, Accuracy: 0.1853


Epoch 2/100, Loss: 1.6402, Accuracy: 0.4352


Epoch 3/100, Loss: 1.2796, Accuracy: 0.5596


Epoch 4/100, Loss: 1.0610, Accuracy: 0.6446


...

Epoch 11/100, Loss: 2.7878, Accuracy: 0.1388


Epoch 12/100, Loss: 2.7847, Accuracy: 0.1395


Epoch 13/100, Loss: 2.7799, Accuracy: 0.1405


Epoch 14/100, Loss: 2.7750, Accuracy: 0.1416


Epoch 15/100, Loss: 2.7723, Accuracy: 0.1397
Early stopping triggered at epoch 15.
